In [1]:
import lightgbm
import numpy as np
from ollama import chat
from sentence_transformers import SentenceTransformer

/Users/pedrotajia/ENTER/envs/AISC_B_project/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/pedrotajia/ENTER/envs/AISC_B_project/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/Users/pedrotajia/ENTER/envs/AISC_B_project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Create a SentenceTransformer instance with the stella_en_1.5B_v5 model
word_embedding = SentenceTransformer(
    "dunzhang/stella_en_1.5B_v5",     # The name/identifier of the Hugging Face model 
    device="mps",                     # Use Apple's Metal Performance Shaders (MPS) for GPU acceleration on Apple Silicon
    config_kwargs={"use_memory_efficient_attention": False},  # Disable memory-efficient attention to ensure compatibility
    trust_remote_code=True            # Allow execution of custom code from the model's repository
)


In [9]:
class Text2Embedding():
    def __init__(self, model):
        """
        The constructor takes in a text-embedding model object (e.g., 
        a SentenceTransformer instance or any other encoding model).
        
        :param model: A text embedding model with a .encode() method.
        """
        self.model = model

    def sentence2vector(self, sentences, name='product_description'):
        """
        Encodes a given text (or list of texts) into an embedding.
        
        :param sentences: The text input(s) to encode.
        :param name: Optional identifier for debugging/logging purposes.
        :return: The raw embedding vector (or a list of vectors).
        """
        print(f"Encoding sentences for feature '{name}'...")
        vector = self.model.encode(sentences)
        return vector

    def transform(self, input):
        """
        A convenience method that calls sentence2vector() and reshapes 
        the resulting embedding(s) to a fixed shape of (1, 1024). 
        This implies the model outputs a 1024-dimensional embedding.
        
        :param input: The text input to encode.
        :return: A (1, 1024) NumPy array representing the single text embedding.
        """
        embedding = self.sentence2vector(input)
        return embedding.reshape(1, 1024)


In [10]:
# Load the model from 'model_gbm.txt' file

gbm = lightgbm.Booster(model_file='model_gbm.txt')

In [15]:

# Define the initial description text
example = "Authentic vintage Chanel made out of luxurious black lambskin. Featuring gold CC closure. Size W: 25cm H: 17cm Size D: 2cm. Shoulder height: 94cm. Inside lining has been fully replaced. Comes with ribbon."
# Transform the text into an embedding 
transform_embedding = Text2Embedding(word_embedding)
example_embedding = transform_embedding.transform(example)

# Define a threshold for the score and the maximum number of iterations
score_threshold = 0.9
max_iterations = 6

# Use a GBM model to predict the "score" for the current description
best_score = gbm.predict(example_embedding)
best_score = float(best_score[0])

# Keep track of the first (original) description and its score
first_example = example
first_score = best_score

# Start iterating until we meet our threshold or reach the max_iterations
i = 1
while best_score < score_threshold and i <= max_iterations:
    # Prepare the prompt as messages for the chat model
    #  - The system message instructs the model to improve the description
    #  - The user message includes the current score and the text to improve
    messages = [
        {
            "role": "system",
            "content": (
                "You task is to improve the description based on a 'score', "
                "bigger 'score' is better, also keep all the essential information "
                "(sizes, colors, etc) anything especific. "
                "Note just return the 'Description'"
            )
        },
        {
            "role": "user",
            "content": f"Score: {best_score:.2f} | Description: {example} "
        },
    ]

    # Call the chat model to get a new/improved description
    response = chat(model="llama3.2", messages=messages)
    
    # Extract the new description from the response
    # Assumes the response content is in the format: "Description: ..."
    example = response['message']['content'].split(':')[-1]

    # Transform the new description to get its embedding
    example_embedding = transform_embedding.transform(example)
    
    # Predict the score of this new description
    new_score = gbm.predict(example_embedding)
    new_score = float(new_score[0])

    # Print debug information: how the score changed from old to new
    print(f"Iteration {i}: Old Score={best_score:.4f}, New Score={new_score:.4f}")

    # If the new score is better, update our best_score and best_text
    if new_score > best_score:
        best_score = new_score
        best_text = example

    i += 1  # Move to the next iteration

# Once we exit the loop, track the final score
last_score = best_score

# Calculate the percentage improvement over the first/original score
percent_of_change = (((last_score - first_score) / first_score) * 100)

# Print out the old and best descriptions with their corresponding scores
print(f'\nOld description: {first_example}')
print(f"\nBest description: {best_text}")
print(f"\nBest score: {best_score}")
print(f"\nPercent of improvment: {percent_of_change:.3f}%")



Encoding sentences for feature 'product_description'...
Encoding sentences for feature 'product_description'...
Iteration 1: Old Score=0.4462, New Score=-0.0014
Encoding sentences for feature 'product_description'...
Iteration 2: Old Score=0.4462, New Score=0.2919
Encoding sentences for feature 'product_description'...
Iteration 3: Old Score=0.4462, New Score=0.2602
Encoding sentences for feature 'product_description'...
Iteration 4: Old Score=0.4462, New Score=0.2523
Encoding sentences for feature 'product_description'...
Iteration 5: Old Score=0.4462, New Score=-0.0596
Encoding sentences for feature 'product_description'...
Iteration 6: Old Score=0.4462, New Score=-0.0444

Old description: ezrxghgvhg good not yttcrtet

Best description: 
This exquisite handbag boasts a majestic shoulder height of 94cm, perfectly proportioned for daily use. Its timeless design has been meticulously restored to perfection, featuring a rich, dark brown leather exterior that showcases a striking contrast